scratch environment
===================

In [ ]:
import org.lamastex.spark.trendcalculus._
import spark.implicits._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import java.sql.Timestamp
import org.apache.spark.sql.expressions._

  

>     import org.lamastex.spark.trendcalculus._
>     import spark.implicits._
>     import org.apache.spark.sql._
>     import org.apache.spark.sql.functions._
>     import java.sql.Timestamp
>     import org.apache.spark.sql.expressions._

In [ ]:
val oilDS = spark.read.fx1m("dbfs:/FileStore/shared_uploads/fabiansi@kth.se/*csv.gz").toDF.withColumn("ticker", lit("BCOUSD")).select($"ticker", $"time" as "x", $"close" as "y").as[TickerPoint].orderBy("time")

// store loaded data as temp view
// oilDS.createOrReplaceTempView("temp")

  

>     oilDS: org.apache.spark.sql.Dataset[org.lamastex.spark.trendcalculus.TickerPoint] = [ticker: string, x: timestamp ... 1 more field]

In [ ]:
//add additional usefull columns to the dataframe

import org.apache.spark.sql.expressions._

val windowSpec = Window.orderBy("x")

val oilDS2 = oilDS
.withColumn("diff_close", $"y" - when((lag("y", 1).over(windowSpec)).isNull, 0).otherwise(lag("y", 1).over(windowSpec)))
.withColumn("index", row_number().over(windowSpec))


// store loaded data as temp view
oilDS2.createOrReplaceTempView("temp")

  

>     import org.apache.spark.sql.expressions._
>     windowSpec: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@7206bf30
>     oilDS2: org.apache.spark.sql.DataFrame = [ticker: string, x: timestamp ... 3 more fields]

In [ ]:
'''
cell is used for testing pyspark syntax only
'''

import datetime
import numpy as np


oilDS_py = spark.table("temp") # data: Row(ticker=u'BCOUSD', x=datetime.datetime(2010, 11, 14, 20, 21), y=86.74), ...
print(oilDS_py.take(10))
start = datetime.datetime(2010, 11, 14, 20, 19)
end = datetime.datetime(2010, 11, 14, 20, 32)

# oilDS_py.filter(oilDS_py.index.between(3, 6))
# np.array(oilDS_py.filter(oilDS_py["x"] > start).filter(oilDS_py["x"] <= end).select('y').collect())

np.array(oilDS_py.where(oilDS_py.index == 5).select('diff_close').collect()).item()

  

>     [Row(ticker=u'BCOUSD', x=datetime.datetime(2010, 11, 14, 20, 15), y=86.74, diff_close=86.74, index=1), Row(ticker=u'BCOUSD', x=datetime.datetime(2010, 11, 14, 20, 17), y=86.75, diff_close=0.010000000000005116, index=2), Row(ticker=u'BCOUSD', x=datetime.datetime(2010, 11, 14, 20, 18), y=86.76, diff_close=0.010000000000005116, index=3), Row(ticker=u'BCOUSD', x=datetime.datetime(2010, 11, 14, 20, 19), y=86.74, diff_close=-0.020000000000010232, index=4), Row(ticker=u'BCOUSD', x=datetime.datetime(2010, 11, 14, 20, 21), y=86.74, diff_close=0.0, index=5), Row(ticker=u'BCOUSD', x=datetime.datetime(2010, 11, 14, 20, 24), y=86.75, diff_close=0.010000000000005116, index=6), Row(ticker=u'BCOUSD', x=datetime.datetime(2010, 11, 14, 20, 26), y=86.77, diff_close=0.01999999999999602, index=7), Row(ticker=u'BCOUSD', x=datetime.datetime(2010, 11, 14, 20, 27), y=86.75, diff_close=-0.01999999999999602, index=8), Row(ticker=u'BCOUSD', x=datetime.datetime(2010, 11, 14, 20, 28), y=86.79, diff_close=0.04000000000000625, index=9), Row(ticker=u'BCOUSD', x=datetime.datetime(2010, 11, 14, 20, 32), y=86.81, diff_close=0.01999999999999602, index=10)]
>     Out[1]: 0.0

In [ ]:
import gym
import math
import numpy as np

PENALTY = 1  # 0.999756079

class MarketEnv(gym.Env):
    def __init__(self, full_data, start_date, end_date, scope=60, cumulative_reward=False):
        self.cumulative_reward = cumulative_reward
        self.actions = ["LONG", "SHORT"] # pass not required, if already owned, buy is hold
        self.action_space = gym.spaces.Discrete(len(self.actions))
        self.state = None
        self.time_range = full_data.filter(full_data["x"] > start_date).filter(full_data["x"] <= end_date)

        self.scope = scope
        self.time_index = 0
        self.reset()
    
    def step(self, action):
        if self.done:
            return self.state, self.reward, self.done, {}
    
        self.reward = 0
        if self.actions[action] == "LONG":
            if sum(self.boughts) < 0:
                for b in self.boughts:
                    self.reward += -(b + 1) 
                if self.cumulative_reward:
                    self.reward = self.reward / max(1, len(self.boughts))
                self.boughts = []
            self.boughts.append(1.0)
        elif self.actions[action] == "SHORT":
            if sum(self.boughts) > 0:
                for b in self.boughts:
                    self.reward += b - 1
                if self.cumulative_reward:
                    self.reward = self.reward / max(1, len(self.boughts))
                self.boughts = []
            self.boughts.append(-1.0)
        else:
            raise ValueError

        print(0)
        diff_close = np.array(self.time_range.where(self.time_range.index == self.time_index).select('diff_close').collect()).item()

        print(1)
        for i in range(len(self.boughts)):
            self.boughts[i] = self.boughts[i] * PENALTY * (1 + diff_close * (-1 if sum(self.boughts) < 0 else 1))

        print(2)
        self.define_state()
        self.time_index += 1
        if self.time_index > self.time_range.count() - self.scope:
            self.done = True

        print(3)
        if self.done:
            for b in self.boughts:
                self.reward += (b * (1 if sum(self.boughts) > 0 else -1)) - 1
            if self.cumulative_reward:
                self.reward = self.reward / max(1, len(self.boughts))
            self.boughts = []

        return self.state, self.reward, self.done
  
    def reset(self):
        self.boughts = []
        self.done = False
        self.reward = 0 
        self.time_index = self.scope
        self.define_state()

        return self.state

    def define_state(self):
        print("call define_state")
        tmpState = []

        budget = (sum(self.boughts) / len(self.boughts)) if len(self.boughts) > 0 else 1.
        size = math.log(max(1., len(self.boughts)), 100)
        position = 1. if sum(self.boughts) > 0 else 0.
        tmpState.append([[budget, size, position]])

        # need to change here!! 
        df_back = self.time_range.filter(self.time_range.index.between(self.time_index, self.time_index + self.scope - 1))
        # TODO check if we go out of range

        np_back = np.array(df_back.select('diff_close').collect())
        # print('[!!] {}'.format(np_back))

        tmpState.append(np_back)

        tmpState = [np.array(i) for i in tmpState]
        self.state = tmpState

    def seed(self):
        pass
    

In [ ]:
start = datetime.datetime(2010, 11, 14, 20, 19)
end = datetime.datetime(2012, 11, 14, 20, 32)

env = MarketEnv(oilDS_py, start, end)

print(env.step(0))

  

>     ([array([[ 1.,  0.,  1.]]), array([[ 0.  ],
>            [ 0.01],
>            [ 0.  ],
>            [ 0.  ],
>            [-0.01],
>            [ 0.01],
>            [ 0.04],
>            [ 0.01],
>            [-0.01],
>            [-0.03],
>            [-0.01],
>            [ 0.  ],
>            [-0.01],
>            [ 0.01],
>            [ 0.  ],
>            [-0.01],
>            [-0.01],
>            [-0.03],
>            [ 0.  ],
>            [ 0.03],
>            [ 0.02],
>            [ 0.05],
>            [ 0.07],
>            [ 0.01],
>            [ 0.01],
>            [ 0.  ],
>            [ 0.  ],
>            [-0.01],
>            [-0.01],
>            [ 0.01],
>            [-0.05],
>            [-0.02],
>            [-0.02],
>            [-0.01],
>            [ 0.02],
>            [-0.01],
>            [ 0.02],
>            [-0.01],
>            [-0.01],
>            [-0.06],
>            [-0.02],
>            [ 0.  ],
>            [ 0.  ],
>            [-0.01],
>            [ 0.  ],
>            [ 0.04],
>            [-0.01],
>            [-0.02],
>            [ 0.  ],
>            [ 0.02],
>            [ 0.01],
>            [-0.01],
>            [ 0.  ],
>            [-0.02],
>            [ 0.  ],
>            [-0.02],
>            [ 0.01],
>            [-0.01],
>            [ 0.  ],
>            [-0.03]])], 0, False)

In [ ]:
print(env.step(0))

  

>     ([array([[ 1.01    ,  0.150515,  1.      ]]), array([[ 0.01],
>            [ 0.  ],
>            [ 0.  ],
>            [-0.01],
>            [ 0.01],
>            [ 0.04],
>            [ 0.01],
>            [-0.01],
>            [-0.03],
>            [-0.01],
>            [ 0.  ],
>            [-0.01],
>            [ 0.01],
>            [ 0.  ],
>            [-0.01],
>            [-0.01],
>            [-0.03],
>            [ 0.  ],
>            [ 0.03],
>            [ 0.02],
>            [ 0.05],
>            [ 0.07],
>            [ 0.01],
>            [ 0.01],
>            [ 0.  ],
>            [ 0.  ],
>            [-0.01],
>            [-0.01],
>            [ 0.01],
>            [-0.05],
>            [-0.02],
>            [-0.02],
>            [-0.01],
>            [ 0.02],
>            [-0.01],
>            [ 0.02],
>            [-0.01],
>            [-0.01],
>            [-0.06],
>            [-0.02],
>            [ 0.  ],
>            [ 0.  ],
>            [-0.01],
>            [ 0.  ],
>            [ 0.04],
>            [-0.01],
>            [-0.02],
>            [ 0.  ],
>            [ 0.02],
>            [ 0.01],
>            [-0.01],
>            [ 0.  ],
>            [-0.02],
>            [ 0.  ],
>            [-0.02],
>            [ 0.01],
>            [-0.01],
>            [ 0.  ],
>            [-0.03],
>            [ 0.01]])], 0, False)

  

reinforcement learning part
===========================

-   see
    https://dbc-635ca498-e5f1.cloud.databricks.com/?o=445287446643905\#notebook/4201196137758409/command/4201196137758420

In [ ]:
class ExperienceReplay:
    def __init__(self, max_memory=100, discount=.9):
        self.max_memory = max_memory
        self.memory = list()
        self.discount = discount

    def remember(self, states, done):
        self.memory.append([states, done])
        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def get_batch(self, model, batch_size=10):
        len_memory = len(self.memory)
        num_actions = model.output_shape[-1]
    
        env_dim = self.memory[0][0][0].shape[1]
        inputs = np.zeros((min(len_memory, batch_size), env_dim, 1))
        targets = np.zeros((inputs.shape[0], num_actions))
        for i, idx in enumerate(np.random.randint(0, len_memory, size=inputs.shape[0])):
            state_t, action_t, reward_t, state_tp1 = self.memory[idx][0]
            done = self.memory[idx][1]

            inputs[i:i+1] = state_t
            # There should be no target values for actions not taken.
            targets[i] = model.predict(state_t)[0]
            Q_sa = np.max(model.predict(state_tp1)[0])
            if done:  # if done is True
                targets[i, action_t] = reward_t
            else:
                # reward_t + gamma * max_a' Q(s', a')
                targets[i, action_t] = reward_t + self.discount * Q_sa
        return inputs, targets

In [ ]:
import json
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Conv1D, MaxPool1D, Flatten
from keras.optimizers import sgd
import collections
import datetime

epsilon = .1  # exploration
num_actions = 2  # [long, short]
epoch = 400
max_memory = 500
hidden_size = 128
batch_size = 64
sequence_scope = 60

input_shape = (batch_size, sequence_scope, 1)

# Create Q Network (s,a --> Q)
model = Sequential()
model.add(Conv1D(32, (5), strides=2, input_shape=input_shape[1:], activation='relu'))
model.add(MaxPool1D(pool_size=2, strides=1))
model.add(Conv1D(32, (5), strides=1, activation='relu'))
model.add(MaxPool1D(pool_size=2, strides=1))
model.add(Flatten())
model.add(Dense(hidden_size, activation='relu'))
model.add(Dense(num_actions))
model.compile(loss='mse', optimizer='adam')

# Define train interval
start = datetime.datetime(2010, 11, 14, 20, 19)
end = datetime.datetime(2010, 12, 14, 20, 32)

print("initialize MarketEnv")
env = MarketEnv(oilDS_py, start, end, scope=sequence_scope, cumulative_reward=False)

# Initialize experience replay object
print("initialize Experience Replay")
exp_replay = ExperienceReplay(max_memory=max_memory)

# Train
rewards = []
for e in range(epoch):
    loss = 0.
    reward_sum = 0.
    print("reset")
    state = env.reset()
    input_t = state[1].reshape(1, sequence_scope, 1) # can't this be moved into the Environment class?
    
    counter = 0
    done = False
    # Start an episode
    while not done:
        counter += 1
        
        input_tm1 = input_t
        # Epsilon Greedy step
        if np.random.rand() <= epsilon:
            # Take randon action
            action = np.random.randint(0, num_actions, size=1)
        else:
            # Take action that maximizes Q value
            print("predict")
            q = model.predict(input_tm1)
            action = np.argmax(q[0])

        # apply action, get rewards and new state
        print("step")
        state, reward, done = env.step(action)
        input_t = state[1].reshape(1, sequence_scope, 1) # can't this be moved into the Environment class?
        reward_sum += reward
          
        # store experience
        print("remember")
        exp_replay.remember([input_tm1, action, reward, input_t], done)

        # adapt model
        print("get batch")
        inputs, targets = exp_replay.get_batch(model, batch_size=batch_size)

        print("train on batch")
        loss += model.train_on_batch(inputs, targets)

    print("Epoch {:03d}/{:d} | Loss {:.4f} | Average Reward {:.4f}".format(e, epoch - 1, loss, reward_sum/counter))

  

>     initialize MarketEnv
>     call define_state
>     initialize ExpRepl
>     reset
>     call define_state
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state
>     3
>     remember
>     get batch
>     train on batch
>     predict
>     step
>     0
>     1
>     2
>     call define_state

  

read data as structures stream
==============================

In [ ]:
val oil_path = "dbfs:/FileStore/shared_uploads/fabiansi@kth.se/*csv.gz"

val input = spark
  .readStream
  .format("delta")
  .load(oil_path)
  .as[TickerPoint]

In [ ]:
joinedDS = spark.read.parquet("dbfs:/FileStore/shared_uploads/fabiansi@kth.se/joinedDSWithMaxRev").orderBy("x")